# LangChain: Evaluation
## Outline:
- Example generation
- Manual evaluation (and debuging)
- LLM-assisted evaluation

In [5]:
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.indexes import VectorstoreIndexCreator

In [6]:
file = 'data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
embedding = OllamaEmbeddings(model="mxbai-embed-large:latest")

In [7]:
index = VectorstoreIndexCreator(
    embedding=embedding,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [8]:
llm = ChatOllama(model = "llama3.1",
                temperature = 0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

# Coming up with test datapoints

In [10]:
data = loader.load()
data[10]

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [11]:
data[11]

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

# Hard-coded examples

In [12]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# LLM-Generated examples

In [13]:
from langchain.evaluation.qa import QAGenerateChain

In [14]:
example_gen_chain = QAGenerateChain.from_llm(ChatOllama(model='llama3.1'))

In [15]:
#Generate examples with llm like above
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/media/ali-hamza/74B82A3BB829FBEC1/AI/Langchain/langchainDeepLearning/venv/lib/python3.12/site-packages/langchain/chains/llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [16]:
new_examples[0]

{'qa_pairs': {'query': 'What is the recommended method for ordering shoes if you have a half size that is not offered?',
  'answer': 'Order up to the next whole size.'}}

In [17]:
new_examples[1]

{'qa_pairs': {'query': 'Where is the Recycled Waterhog Dog Mat, Chevron Weave manufactured?',
  'answer': 'Made in the USA.'}}

In [18]:
new_examples[2]

{'qa_pairs': {'query': "What type of fabric does this toddler's two-piece swimsuit have, which keeps its shape and resists snags?",
  'answer': 'Four-way-stretch and chlorine-resistant fabric.'}}

In [19]:
new_examples[3]

{'qa_pairs': {'query': 'What is the primary material used in the body of the swimtop, aside from Lycra spandex?',
  'answer': 'Recycled nylon (82%)'}}

In [20]:
new_examples[4]

{'qa_pairs': {'query': 'What percentage of nylon is used in the construction of the EcoFlex 3L Storm Pants, excluding trim?',
  'answer': '100%'}}

In [21]:
data[0]

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [22]:
data[0]

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [23]:
examples += new_examples

In [24]:
qa.run(examples[0]["query"])

/tmp/ipykernel_14129/1223946598.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(examples[0]["query"])




> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.'

# For further analysis we enable debuge

In [25]:
import langchain
langchain.debug = True

In [27]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-

'Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.'

# Lets check this will all examples with LLM

In [35]:
langchain.debug = False

In [37]:
for e in examples:
    print(e)


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'answer': 'Yes'}
{'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?', 'answer': 'The DownTek collection'}
{'qa_pairs': {'query': 'What is the recommended method for ordering shoes if you have a half size that is not offered?', 'answer': 'Order up to the next whole size.'}}
{'qa_pairs': {'query': 'Where is the Recycled Waterhog Dog Mat, Chevron Weave manufactured?', 'answer': 'Made in the USA.'}}
{'qa_pairs': {'query': "What type of fabric does this toddler's two-piece swimsuit have, which keeps its shape and resists snags?", 'answer': 'Four-way-stretch and chlorine-resistant fabric.'}}
{'qa_pairs': {'query': 'What is the primary material used in the body of the swimtop, aside from Lycra spandex?', 'answer': 'Recycled nylon (82%)'}}
{'qa_pairs': {'query': 'What percentage of nylon is used in the construction of the EcoFlex 3L Storm Pants, excluding trim?', 'answer': '100%'}}


In [46]:
flattened_examples = []

for item in examples:
    if 'query' in item:
        flattened_examples.append({'query': item['query'], 'answer': item.get('answer', '')})
    elif 'qa_pairs' in item and 'query' in item['qa_pairs']:
        flattened_examples.append({'query': item['qa_pairs']['query'], 'answer': item['qa_pairs'].get('answer', '')})

In [48]:
for e in flattened_examples:
    print(e)

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'answer': 'Yes'}
{'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?', 'answer': 'The DownTek collection'}
{'query': 'What is the recommended method for ordering shoes if you have a half size that is not offered?', 'answer': 'Order up to the next whole size.'}
{'query': 'Where is the Recycled Waterhog Dog Mat, Chevron Weave manufactured?', 'answer': 'Made in the USA.'}
{'query': "What type of fabric does this toddler's two-piece swimsuit have, which keeps its shape and resists snags?", 'answer': 'Four-way-stretch and chlorine-resistant fabric.'}
{'query': 'What is the primary material used in the body of the swimtop, aside from Lycra spandex?', 'answer': 'Recycled nylon (82%)'}
{'query': 'What percentage of nylon is used in the construction of the EcoFlex 3L Storm Pants, excluding trim?', 'answer': '100%'}


In [49]:
predictions = qa.apply(flattened_examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [50]:
from langchain.evaluation.qa import QAEvalChain

In [51]:
llm = ChatOllama(temperature=0, model = "llama3.1")
eval_chain = QAEvalChain.from_llm(llm)

In [52]:
graded_outputs = eval_chain.evaluate(flattened_examples, predictions)


In [55]:
print(graded_outputs)

[{'results': 'GRADE: CORRECT'}, {'results': 'GRADE: CORRECT'}, {'results': 'GRADE: INCORRECT'}, {'results': 'GRADE: CORRECT'}, {'results': 'GRADE: CORRECT'}, {'results': 'GRADE: CORRECT'}, {'results': 'GRADE: CORRECT'}]


In [58]:
for i, eg in enumerate(flattened_examples):
    print(f"Examples ({i}):")
    print(f"Question: " + predictions[i]['query'])
    print(f"Real Answer: " + predictions[i]['answer'])
    print(f"Predicted Answer: " + predictions[i]['result'])
    print(f"Predicted Grade: " + graded_outputs[i]['results'])
    print()

Examples (0):
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.
Predicted Grade: GRADE: CORRECT

Examples (1):
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from our DownTek collection.
Predicted Grade: GRADE: CORRECT

Examples (2):
Question: What is the recommended method for ordering shoes if you have a half size that is not offered?
Real Answer: Order up to the next whole size.
Predicted Answer: According to the context, for the Men's Leather Lace-Ups by L.L.Bean, it is recommended to "order 1½ sizes down" when ordering half sizes. This is stated in the Size & Fit section of the description.

However, I don't see this information provided for the Women's Katahdin Hiking Sneakers, Nubuck Mesh.
Predicted Grade: GR